In [1]:
import cv2
import numpy as np
import time
import os
import ipynb.fs.full.handTrackingModule  as htm

In [2]:
folderpath ="header"
myList = os.listdir(folderpath)
overlayList =[]
for impath in myList:
    image = cv2.imread(f'{folderpath}/{impath}')
    overlayList.append(image)
print(len(overlayList))
header = overlayList[0]

4


In [3]:
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
detector = htm.handDetector(detectionCon=0.6)
imgCanvas = np.zeros((720,1280,3),np.uint8)
xp,yp =0,0
drawColor =(255,0,0)
while True:
    # 1 Import image
    success,img = cap.read()
    img = cv2.flip(img,1)
    
    # Find Hand landmarks
    img = detector.findHands(img)
    lmList = detector.findPositions(img,draw=False)
    
    if len(lmList)!=0:
        
        # tp of index and middle finger
        x1,y1 = lmList[8][1:]
        x2,y2 = lmList[12][1:] 
        
        # chk which fingers are up
        fingers = detector.fingersUp()
        # print(fingers)
        
        #  if selection mode - two fingers are up
        if fingers[1] and fingers[2]:
            xp,yp=0,0
            if y1<125:
                if 0<x1<300:
                    header = overlayList[0]
                    drawColor = (255,0,0)          
                elif 320<x1<620:
                    header = overlayList[1]
                    drawColor = (0,0,255)
                elif 640<x1<940:
                    header = overlayList[2]
                    drawColor = (0,255,0)
                elif 960<x1<1260:
                    header = overlayList[3]
                    drawColor = (0,0,0)
            cv2.circle(img,((x1+x2)//2,(y1+y2)//2),20,drawColor,cv2.FILLED)                    
        # if Drawing mode - Index finger is up
        if fingers[1] and (fingers[2]==False):
            cv2.circle(img,(x1,y1),10,drawColor,cv2.FILLED)
            if xp==0 and yp==0:
                xp,yp=x1,y1
            if drawColor ==(0,0,0):
                cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,50)
            else:
                cv2.line(img,(xp,yp),(x1,y1),drawColor,15)
                cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,15)
            xp,yp=x1,y1
            #print("Drawing mode")
    imgGray = cv2.cvtColor(imgCanvas,cv2.COLOR_BGR2GRAY)
    _,imgInv =cv2.threshold(imgGray,50,255,cv2.THRESH_BINARY_INV)
    imgInv =cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img,imgInv)
    img = cv2.bitwise_or(img,imgCanvas)
    # Setting the header
    h,w,c = header.shape
    img[0:h,0:w]=header
    #img = cv2.addWeighted(img,0.5,imgCanvas,0.5,0)
    cv2.imshow("image",img)
    #cv2.imshow("canvas",imgCanvas)
    k =cv2.waitKey(1)
    if k==27:
        break;
cap.release()
cv2.destroyAllWindows()